In [1]:
import numpy as np
from scipy.special import factorial, factorial2
from sympy.physics.wigner import wigner_9j, wigner_6j, wigner_3j
from itertools import product

from moshinsky_way import set_moshinsky_brackets

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# Test that our Moshinsky brackets are correct (quite incomplete but oh well)

loaded_brackets = set_moshinsky_brackets(10, 10)

si = np.sqrt(1/2)
co = -si

def A(n, l):
    return (-1)**n * (factorial2(2*n) * factorial2(2*n + 2*l + 1))**(-1/2)


In [3]:

# l1 = 0
# l2 = 0
# n1 = 0
# n2 = 0
# lamb = 0

count = 0
for l1, l2 in product(range(0, 4), repeat=2):
    for n1, n2 in product(range(0, 4), repeat=2):
        for lamb in range(np.abs(l1-l2), l1+l2+1):
        
            Nq = 2*n1 + l1 + 2*n2 + l2

            # if (l1 + l2) % 2 != 1:
            #     continue

            if (Nq - lamb) % 2 != 0:
                continue

            n_prime = (Nq - lamb)//2

            if 2*n_prime + lamb != Nq:
                print("WTF")

            bracket = float(co**(2*n1 + l1) * si**(2*n2 + l2) * (-1)**lamb * ((2*l1 + 1) * (2*l2 + 1))**(1/2) *\
                        wigner_3j(l1, l2, lamb, 0, 0, 0).evalf() * A(n1, l1) * A(n2, l2) / A(n_prime, lamb))


            loaded_bracket = loaded_brackets[n_prime, lamb, 0, 0, n1, l1, l2, lamb]
            loaded_bracket_2 = loaded_brackets[n1, l1, n2, l2, n_prime, lamb, 0, lamb]

            #print(bracket, loaded_bracket)

            if not np.isclose(loaded_bracket, loaded_bracket_2, atol=1e-3):
                print("WTF")
                print(loaded_bracket, loaded_bracket_2)

            if not np.isclose(bracket, loaded_bracket, atol=1e-3):
                print("Error in Moshinsky brackets:", n1, l1, n2, l2, n_prime, lamb)
                print(bracket, loaded_bracket)
                #print(A(n1, l1), A(n2, l2), A(n_prime, lamb), wigner_3j(l1, l2, lamb, 0, 0, 0).evalf())
                print("even" if (l1 + l2) % 2 == 0 else "odd")
                print(Nq)
            
            count += 1

print("COUNT:", count)

COUNT: 480
